In [ ]:
import pandas as pd 
import ipywidgets as widgets
import datetime
import sqlalchemy
import psycopg2
import golf_functions

This notebook is used to load all info about a played round of golf. This info will be loaded into round_info and hole_scores. 

For these tables you will need:
- course/tee played
- date played
- weather played in
- number of holes played
- for each hole played:
    - score 
    - fairway in regulation
    - fairway miss side
    - green in regulation
    - green miss side
    - number of putts
    - other notes

In [ ]:
course_select_query = '''
Select ci.course_id
    ,Max(ci.course_version_id) as course_version_id
    ,cn.course_name
    ,ci.tee
from "Courses"."course_info" ci 
left join "Courses"."course_name" cn on cast(ci.course_id as int) = cn.course_id
group by ci.course_id, ci.tee, cn.course_name
'''

In [ ]:
course_tee_select = golf_functions.pull_from_sql(course_select_query)
course_tee_select['course_tee'] = course_tee_select['course_name'] + ' - ' + course_tee_select['tee']

In [ ]:
# selecting course/tee
course_list = list(course_tee_select['course_tee'])
course_list.sort()
ct_select = widgets.Dropdown(options = course_list)
ct_select

In [ ]:
course_select = course_tee_select[course_tee_select['course_tee'] == ct_select.value]
course_select.reset_index(inplace=True)

In [ ]:
# labels and widgets for entry about info of the round
weather_types = ['Sun', 'Rain', 'Snow', 'Light Breeze', 'Wind', 'Cold', 'Hot', 'NA']
weather_types.sort()
date_lbl = widgets.Label('Entry_date')
date_ent = widgets.DatePicker()
weather_lbl = widgets.Label('Weather While Playing')
weather_ent1 = widgets.Dropdown(options = weather_types)
weather_ent2 = widgets.Dropdown(options = weather_types)
display(date_lbl)
display(date_ent)
display(weather_lbl)
display(weather_ent1)
display(weather_ent2)

In [ ]:
round_info = course_select.loc[:,('course_id', 'course_version_id', 'tee')]
round_info['date_played'] = date_ent.value.strftime('%Y-%m-%d')
if weather_ent2.value != 'NA':
    weather_entry = weather_ent1.value +'/'+ weather_ent2.value
else:
    weather_entry = weather_ent1.value
round_info['weather'] = weather_entry
# previewing round_info
round_info

In [ ]:
# selecting round length
round_length  = ['Full 18', 'Front 9', 'Back 9']
length_select = widgets.Dropdown(options = round_length)
length_select

In [ ]:
# pulling hole info about holes that were played
if length_select.value == 'Full 18':
    holes_select = '1 and 18'
    hole_range = [1,19]
elif length_select.value == 'Front 9':
    holes_select = '1 and 9'
    hole_range = [1,10]
elif length_select.value == 'Back 9':
    holes_select = '10 and 18'
    hole_range = [10,19]
holes_played_query = '''
Select * from "Courses"."hole_info"
where course_id = {0} and tee = {1} and cast(hole as int) between {2};
'''.format("'"+course_select['course_id'][0]+"'",
            "'"+course_select['tee'][0]+"'",
            holes_select)
hole_played = golf_functions.pull_from_sql(holes_played_query)
hole_played

In [ ]:
#pulling notes and creating lists for entry selection
note_query = '''
Select note, note_abbv from "Scores"."note_definations"
'''
notes = golf_functions.pull_from_sql(note_query)
notes_name = list(notes['note'])
notes_name.sort()
miss_list = ['Y', 'N', 'NA']
miss_way_list = ['Right', 'Left', 'Short', 'Long', 'NA']

In [ ]:
# creating labels and widgets for data entry
# used globals to create variables with different names within a four loop. 
for h in range(0,hole_played.shape[0]):
    temp_df = hole_played.loc[h]
    globals()['header_%s' % h + '_lbl'] = widgets.Label('Hole '+temp_df['hole']+ ' Par '+temp_df['par'])
    globals()['score_%s' % h + '_lbl'] = widgets.Label('Score')
    globals()['score_%s' % h + '_ent'] = widgets.IntText(value = int(temp_df['par']))
    globals()['FIR_%s' % h + '_lbl'] = widgets.Label('Fairway in Regulation')
    globals()['FIR_%s' % h + '_ent'] = widgets.Dropdown(options = miss_list, value = None)
    globals()['FIR_ms_%s' % h + '_lbl'] = widgets.Label('Fairway in Regulation Miss Side')
    globals()['FIR_ms_%s' % h + '_ent'] = widgets.Dropdown(options = miss_way_list, value = None)
    globals()['GIR_%s' % h + '_lbl'] = widgets.Label('Green in Regulation')
    globals()['GIR_%s' % h + '_ent'] = widgets.Dropdown(options = miss_list, value = None)
    globals()['GIR_ms_%s' % h + '_lbl'] = widgets.Label('Green in Regulation Miss Side')
    globals()['GIR_ms_%s' % h + '_ent'] = widgets.Dropdown(options = miss_way_list, value = None)
    globals()['putts_%s' % h + '_lbl'] = widgets.Label('Number of Putts')
    globals()['putts_%s' % h + '_ent'] = widgets.IntText()
    globals()['notes_%s' % h + '_lbl'] = widgets.Label('Notes')
    globals()['notes_1_%s' % h + '_ent'] = widgets.Dropdown(options = notes_name, value = 'No Note')
    globals()['notes_2_%s' % h + '_ent'] = widgets.Dropdown(options = notes_name, value = 'No Note')
    globals()['hole_%s' % h + '_ent'] = temp_df['hole']
    globals()['par_%s' % h + '_ent'] = temp_df['par']
    display(globals()['header_%s' % h + '_lbl'])
    display(globals()['score_%s' % h + '_lbl'])
    display(globals()['score_%s' % h + '_ent'])
    display(globals()['FIR_%s' % h + '_lbl'])
    display(globals()['FIR_%s' % h + '_ent'])
    display(globals()['FIR_ms_%s' % h + '_lbl'])
    display(globals()['FIR_ms_%s' % h + '_ent'])
    display(globals()['GIR_%s' % h + '_lbl'])
    display(globals()['GIR_%s' % h + '_ent'])
    display(globals()['GIR_ms_%s' % h + '_lbl'])
    display(globals()['GIR_ms_%s' % h + '_ent'])
    display(globals()['putts_%s' % h + '_lbl'])
    display(globals()['putts_%s' % h + '_ent'])
    display(globals()['notes_%s' % h + '_lbl'])
    display(globals()['notes_1_%s' % h + '_ent'])
    display(globals()['notes_2_%s' % h + '_ent'])

In [ ]:
# creating lists of date entered for dict
hole_list = []
par_list = []
score_list = []
FIR_list = []
FIR_miss_list = []
GIR_list = []
GIR_miss_list = []
putts_list = []
notes_1_list = []
notes_2_list = []
for h in range(0,hole_played.shape[0]):
    hole_list.append(globals().get('hole_%s' % h + '_ent'))
    par_list.append(globals().get('par_%s' % h + '_ent'))
    score_list.append(globals().get('score_%s' % h + '_ent').value)
    FIR_list.append(globals().get('FIR_%s' % h + '_ent').value)
    FIR_miss_list.append(globals().get('FIR_ms_%s' % h + '_ent').value)
    GIR_list.append(globals().get('GIR_%s' % h + '_ent').value)
    GIR_miss_list.append(globals().get('GIR_ms_%s' % h + '_ent').value)
    putts_list.append(globals().get('putts_%s' % h + '_ent').value)
    notes_1_list.append(globals().get('notes_1_%s' % h + '_ent').value)
    notes_2_list.append(globals().get('notes_2_%s' % h + '_ent').value)
# creating dict and df of data entered
hole_played_dict = {
    'hole_number':hole_list,
    'par':par_list,
    'score':score_list,
    'fir':FIR_list,
    'f_miss_side':FIR_miss_list,
    'gir':GIR_list,
    'g_miss_side':GIR_miss_list,
    'putts':putts_list,
    'note_1':notes_1_list,
    'note_2':notes_2_list
}
# updating and creating new columns off of od entered data
hole_scores = pd.DataFrame(hole_played_dict,columns=hole_played_dict.keys())
hole_scores['score_to_par'] = hole_scores['score'] - hole_scores['par'].astype(int)
for col in ['note_1', 'note_2']:
    hole_scores[col] = hole_scores[col].map(pd.Series(notes['note_abbv'].values, index=notes['note']).to_dict())
# first check to make sure data entered correctly
hole_scores

In [ ]:
round_info['date_played'] = pd.to_datetime(round_info['date_played'])
round_info.rename(columns= {'tee':'tee_played'}, inplace=True)
print('Please Check that Round Infromation is Correct')
# double checking round info is correct
round_info

If the Infromation above is correct please run the next cell

In [ ]:
golf_functions.load_sql(round_info,'Scores','round_info')

In [ ]:
# sql query is needed to get round_id that is made when round_info in inserted
round_id_query = '''
Select MAX(round_id) as round_id from "Scores"."round_info"
where course_id = {0} and cast(date_played as date) = {1}
'''.format("'"+round_info['course_id'][0]+"'", "'"+date_ent.value.strftime('%Y-%m-%d')+"'")
round_id_df = golf_functions.pull_from_sql(round_id_query)
hole_scores['round_id'] = round_id_df['round_id'][0].astype(str)
hole_scores.drop('par', axis=1, inplace=True)
print('Please Check the Below Hole Score Infromation')
# double checking entered info is correct
hole_scores

If the Above Infromation is Correct Please Run the Next Cell

In [ ]:
golf_functions.load_sql(hole_scores, 'Scores','hole_scores')